In [ ]:
# TODO
# 1. Add to predict path to leaf + 
# 2. Test +
# 3. Pruning. Max. depth +
# 4. Gini index +
# 5. Comments +
# 6. Continuos features with gini index +
# 7. Continuos features with information gain
# 8. Pruning. Min. samples for splitting +
# 9. Add parallel computing + 
# 10. Why current implementation is much much slower then from sklearn ?

In [1]:
import numpy as np
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor 
from multiprocessing import Pool, cpu_count
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# The following three functions have been moved out of the DecisionTree class so 
# that they can be imported directly. This is necessary to parallelize the process 
# of checking the thresholds of a continuous feature

In [2]:
def get_entropy(y):
    """
    Evaluate entropy for vector with class labels
    Entropy = sum_1_to_C(-p_i*log2(p_i)), where C - number of classes
    
    Argument
        y - array-like variable with class labels
    
    Return
        Entropy of vector y  
    """
    _, counts = np.unique(y, return_counts=True)
    total = np.sum(counts)
    fractions = counts/total
    return -np.sum(fractions*np.log2(fractions))

In [3]:
def get_gini_index_by_feature_value(y):
    """
    Evaluate Gini index for vector with class labels
    Gini_index = 1 - sum_1_to_C(-p_i^2), where C - number of classes
    
    Argument
        y - array-like variable with class labels
    
    Return
        Gini_index of vector y  
    """    
    _, counts = np.unique(y, return_counts=True)
    total = np.sum(counts)
    fractions = counts/total
    return 1-np.sum(fractions**2)

In [4]:
def evaluate_gini_index_for_threhold(data):
    """
    Evaluate Gini index for continuous feature with a given threshold
    
    Gini_index_given threshold = w1 * gini_index_less_or_equal_thr + w2 * gini_index_greater_thr
    
    Argument
        data - tuple, where data[0] - feature values vector, 
        data[1] - vector with class labels, data[2] - splitting threshold
    
    Return
        Gini_index for continuous feature with a given threshold 
    """ 
    X_feature, y, thr = data
    y_by_feature_value = y[X_feature <= thr]
    w1 = len(y_by_feature_value)/len(y)
    less_or_equal = get_gini_index_by_feature_value(y_by_feature_value)
    y_by_feature_value = y[X_feature > thr]
    w2 = len(y_by_feature_value)/len(y)
    greater = get_gini_index_by_feature_value(y_by_feature_value)
    return w1*less_or_equal + w2*greater   

In [65]:
class Node:
    """
    Сlass for storing tree's node data
    """

    def __init__(self, cost, class_, classes=None, classes_distrib=None, childs=None, depth=0):
        self.cost = cost
        self.class_ = class_
        self.split_feature = None
        self.threshold = None
        self.parent_feature_value = None
        self.classes = [] if classes is None else classes
        self.classes_distrib = [] if classes_distrib is None else classes_distrib            
        self.childs = [] if childs is None else childs
        self.depth=depth
    
    def __repr__(self):
        mn = self.depth*3
        return  f"{(mn*' ')}Value of parent's feature: {self.parent_feature_value}\n"\
                f"{(mn*' ')}Node class: {self.class_}\n"\
                f"{(mn*' ')}Node cost: {self.cost}\n"\
                f"{(mn*' ')}Classes in node: {self.classes}\n"\
                f"{(mn*' ')}Classes distribution in node: {self.classes_distrib}\n"\
                f"{(mn*' ')}Is leaf: {('False' if len(self.childs) else 'True')}\n"\
                f"{(mn*' ')}Feature selected fo split: {self.split_feature}\n"\
                f"{(mn*' ')}Threshold splitting: {self.threshold}\n"\
                f"{(mn*' ')}childs:\n{(mn*' ')}{self.childs}\n"
        
class DecisionTree:
    
    """
    Class for fitting Decision tree for classification tasks. The built tree is stored in nodes, which 
    are implemented using class Node. Features can be as categorical as continuous.  
    """
    
    def __init__(self, criterion="gini", max_depth=10, min_samples_for_splitting=2):
        if criterion =="entropy":
            self.__select_feature_function = self.__select_feature_with_max_inform_gain
            self.__evalute_criterion_function = get_entropy
        elif criterion =="gini":
            self.__select_feature_function = self.__select_feature_with_min_gini_index
            self.__evalute_criterion_function = get_gini_index_by_feature_value
        else:
            raise ValueError("Criterion must be 'entropy' or 'gini'")
            
        self.root = None
        self.max_depth = max_depth
        self.min_samples_for_splitting=min_samples_for_splitting
        
#     def __get_entropy(self, y):
#         """
#         Evaluate entropy for vector
#         """
#         _, counts = np.unique(y, return_counts=True)
#         total = np.sum(counts)
#         fractions = counts/total
#         return abs(-np.sum(fractions*np.log2(fractions)))

#     def __get_gini_index_by_feature_value(self, y):
#         _, counts = np.unique(y, return_counts=True)
#         total = np.sum(counts)
#         fractions = counts/total
#         return 1-np.sum(fractions**2)

    def __get_thresholds(self, X_feature, y):
        thresholds = []
        X_feature_sorted = X_feature.sort_values()
        current_target_value = y.loc[X_feature_sorted.index[0]]
        for i, idx in enumerate(X_feature_sorted.index[1:]):
            if np.all(y.loc[idx] != current_target_value):
                current_target_value = y.loc[idx]
                if X_feature_sorted.loc[idx] != X_feature_sorted.loc[X_feature_sorted.index[i]]:
                    thresholds.append((X_feature_sorted.loc[idx] + X_feature_sorted.loc[X_feature_sorted.index[i]])/2)
        return sorted(list(set(thresholds)))

    def __get_gini_index(self, X_feature, y):
        
#         def evaluate_gini_index_for_threhold(self, thr):
# #             print(f"Start thr={thr}")
#             y_by_feature_value = y[X_feature <= thr]
#             w1 = len(y_by_feature_value)/len(y)
#             less_or_equal = self.__get_gini_index_by_feature_value(y_by_feature_value)
#             y_by_feature_value = y[X_feature > thr]
#             w2 = len(y_by_feature_value)/len(y)
#             greater = self.__get_gini_index_by_feature_value(y_by_feature_value)
# #             print(f"Finish thr={thr}")
#             return w1*less_or_equal + w2*greater            
            
        threshold = None
        if X_feature.dtypes == np.int64 or X_feature.dtypes == np.float64:
            gini_index = np.inf
            possible_thresholds = self.__get_thresholds(X_feature, y)
            if not possible_thresholds:
                return np.inf, None
#             print(f"Possible thresholds: {possible_thresholds}")
            data = [(X_feature, y, thr) for thr in possible_thresholds]
            with Pool(cpu_count()-1) as pool:
                gs = pool.map(evaluate_gini_index_for_threhold, data)   
            gini_index = np.min(gs)
            threshold = possible_thresholds[np.argmin(gs)]
#             print(threshold, gini_index)
            
#             for value in possible_thresholds:
#                 y_by_feature_value = y[X_feature <= value]
#                 w1 = len(y_by_feature_value)/len(y)
#                 less_or_equal = self.__get_gini_index_by_feature_value(y_by_feature_value)
#                 y_by_feature_value = y[X_feature > value]
#                 w2 = len(y_by_feature_value)/len(y)
#                 greater = self.__get_gini_index_by_feature_value(y_by_feature_value)
#                 tmp_gini = w1*less_or_equal + w2*greater
#                 if gini_index > tmp_gini:
#                     gini_index = tmp_gini
#                     threshold = value
#             print(threshold, gini_index)            
        else:
            gini_index = 0
            unique_values, unique_counts = np.unique(X_feature, return_counts=True)
#             print(f"Possible values: {unique_values}")
            weights = unique_counts/np.sum(unique_counts)
            for idx, value in enumerate(unique_values):
                y_by_feature_value = y[X_feature == value]
                gini_index +=weights[idx]*get_gini_index_by_feature_value(y_by_feature_value)
        return gini_index, threshold    
    
    def __get_information_gain(self, X_feature, y):
        entropy_before = get_entropy(y)
        entropy_after = 0
        unique_values, unique_counts = np.unique(X_feature, return_counts=True)
        weights = unique_counts/np.sum(unique_counts)
        for idx, value in enumerate(unique_values):
            y_by_feature = y[X_feature == value]
            entropy_after +=weights[idx]*get_entropy(y_by_feature)
        return entropy_before - entropy_after
    
    def __select_feature_with_max_inform_gain(self, X, y):
        igs = []
        for col in X.columns:
            print(f"Checking feature {col}")
            ig = self.__get_information_gain(X[col], y)
            igs.append(ig)
        print(f"Best feature is {X.columns[np.argmax(igs)]}")
        return X.columns[np.argmax(igs)]
    
    def __select_feature_with_min_gini_index(self, X, y):
        gis = []
        with ThreadPoolExecutor(max_workers=len(X.columns)) as executor:
            for col in X.columns:
                print(f"Checking feature {col}")
                gis.append(executor.submit(self.__get_gini_index, X[col], y).result())
        
#         for col in X.columns:
#             print(f"Checking feature {col}")
#             gi, thr = self.__get_gini_index(X[col], y)
#             gis.append((gi, thr))
            
        idx = np.argmin([el[0] for el in gis])
        print(f"Best feature is {X.columns[idx]}")
        return X.columns[idx], gis[idx][1]
    
    def __split_node(self, X, y, depth):
        print("Depth:", depth)
        classes, counts = np.unique(y, return_counts=True)
        cost = self.__evalute_criterion_function(y)
        node = Node(cost=cost, class_=classes[np.argmax(counts)], classes=classes, classes_distrib=counts, depth=depth) 
        if len(classes)==1 or depth>=self.max_depth or np.sum(counts)<self.min_samples_for_splitting:
            return node
        current_feature, threshold = self.__select_feature_function(X, y)
        node.split_feature = current_feature
        node.threshold = threshold
        if threshold is None:
            unique_values = np.unique(X[current_feature])
            for value in unique_values:
                X_after_split = X[X[current_feature]==value]
#                 X_after_split = X_after_split.drop(current_feature, axis=1)
                y_after_split = y[X_after_split.index]
                child = self.__split_node(X_after_split, y_after_split, depth+1)
                child.parent_feature_value = value
                node.childs.append(child)
        else: 
            X_after_split_le = X[X[current_feature]<=threshold]
            y_after_split = y[X_after_split_le.index]
            child = self.__split_node(X_after_split_le, y_after_split, depth+1)
            child.parent_feature_value = None
            node.childs.append(child)
            X_after_split_g = X[X[current_feature]>threshold]
            y_after_split = y[X_after_split_g.index]
            child = self.__split_node(X_after_split_g, y_after_split, depth+1)
            child.parent_feature_value = None
            node.childs.append(child)
        return node
        
    def fit(self, X, y):
        self.root = self.__split_node(X, y, 0)      
    
    def __get_next_node(self, current_node, X):
        splitted_value = X[current_node.split_feature]
        if current_node.threshold is None:
            for child in current_node.childs:
                if child.parent_feature_value == splitted_value:
                    return child
        else:
            return current_node.childs[0] if splitted_value<=current_node.threshold else current_node.childs[1]
        return None
    
    def predict(self, X):
        y, ways = [], [] 
        for i in range(len(X)):
            path = []
            current_node = self.root
            is_leaf = False if len(current_node.childs) else True
            while not is_leaf:
                parent_node = current_node
                current_node = self.__get_next_node(current_node, X.iloc[i])
                if current_node is not None:
                    path.append({"feature": parent_node.split_feature, "value": current_node.parent_feature_value})
                is_leaf = False if (current_node is not None and len(current_node.childs)) else True
            y.append(current_node.class_ if current_node else parent_node.class_)
            ways.append(path)
        return y, ways
        

## Testing on synthetic data

In [37]:
data_dict = {"Outlook":["Sunny", "Sunny", "Overcast", "Rain", "Rain", "Rain", 
                        "Overcast", "Sunny", "Sunny", "Rain", "Sunny", 
                        "Overcast", "Overcast", "Rain" ], 
             "Temp": [30, 30, 30, 20, 10, 10, 10, 20, 10, 20, 20, 20, 30, 20], 
             "Humidity": ["High", "High", "High", "High", "Norm", 
                          "Norm", "Norm", "High", "Norm", "Norm", 
                          "Norm", "High", "Norm", "High"], 
             "Wind": ["Weak", "Strong", "Weak", "Weak", "Strong", "Weak", 
                      "Weak", "Weak", "Weak", "Strong", "Strong", "Strong", 
                      "Weak", "Strong"], 
             "Tennis": ["No", "No", "Yes", "Yes", "Yes", "No", "Yes", "No", 
                        "Yes", "Yes", "Yes", "Yes", "Yes", "No" ]} 
df = pd.DataFrame(data_dict)
df

,Outlook,Temp,Humidity,Wind,Tennis
0,Sunny,30,High,Weak,No
1,Sunny,30,High,Strong,No
2,Overcast,30,High,Weak,Yes
3,Rain,20,High,Weak,Yes
4,Rain,10,Norm,Strong,Yes
5,Rain,10,Norm,Weak,No
6,Overcast,10,Norm,Weak,Yes
7,Sunny,20,High,Weak,No
8,Sunny,10,Norm,Weak,Yes
9,Rain,20,Norm,Strong,Yes


In [67]:
X = df.loc[:,'Outlook':'Wind']
y = df['Tennis']

In [70]:
dt = DecisionTree(max_depth=4, min_samples_for_splitting=2)
dt.fit(X, y)

Depth: 0
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Outlook
Depth: 1
Depth: 1
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Temp
Depth: 2
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Wind
Depth: 3
Depth: 3
Depth: 2
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Humidity
Depth: 3
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Wind
Depth: 4
Depth: 4
Depth: 3
Depth: 1
Checking feature Outlook
Checking feature Temp
Checking feature Humidity
Checking feature Wind
Best feature is Humidity
Depth: 2
Depth: 2


In [21]:
y_hat, explains = dt.predict(X)

In [22]:
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

          No       0.83      1.00      0.91         5
         Yes       1.00      0.89      0.94         9

    accuracy                           0.93        14
   macro avg       0.92      0.94      0.93        14
weighted avg       0.94      0.93      0.93        14



## Testing on real data

### Dataset from here: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package?rvi=1

In [71]:
# Loading dataset and looking at a feature types and number of rows
df = pd.read_csv('weatherAUS.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [72]:
# Looking at the data
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [74]:
# Removing features with many NaN values
df_preprocessing = df.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Date'], axis=1)
# Removing any row with NaN values
df_preprocessing.dropna(inplace=True)
df_preprocessing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112925 entries, 0 to 145458
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       112925 non-null  object 
 1   MinTemp        112925 non-null  float64
 2   MaxTemp        112925 non-null  float64
 3   Rainfall       112925 non-null  float64
 4   WindGustDir    112925 non-null  object 
 5   WindGustSpeed  112925 non-null  float64
 6   WindDir9am     112925 non-null  object 
 7   WindDir3pm     112925 non-null  object 
 8   WindSpeed9am   112925 non-null  float64
 9   WindSpeed3pm   112925 non-null  float64
 10  Humidity9am    112925 non-null  float64
 11  Humidity3pm    112925 non-null  float64
 12  Pressure9am    112925 non-null  float64
 13  Pressure3pm    112925 non-null  float64
 14  Temp9am        112925 non-null  float64
 15  Temp3pm        112925 non-null  float64
 16  RainToday      112925 non-null  object 
 17  RainTomorrow   112925 non-nul

In [26]:
# Splitting on feature matrix (X) and target vector - y
X = df_preprocessing.loc[:, 'Location':'RainToday']
y = df_preprocessing['RainTomorrow']

In [27]:
# We select only columns with numeric types so that we can check the results of current implementation 
# with the implementation from sklearn
numerical_columns = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X[numerical_columns], y, test_size=0.2)

In [29]:
dt = DecisionTree(max_depth=10)
dt.fit(X_train, y_train)

Depth: 0
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Humidity3pm
Depth: 1
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Humidity3pm
Depth: 2
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9

Checking feature Temp3pm
Best feature is Pressure3pm
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is WindGustSpeed
Depth: 10
Depth: 10
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MinTemp
Depth: 10
Depth: 10
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSp

Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Pressure3pm
Depth: 8
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is WindSpeed9am
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Temp3pm
Depth: 10
Depth: 10
Depth: 9
Depth: 6
Checking feature MinTe

Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Pressure9am
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Temp9am
Depth: 10
Depth: 10
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking fe

Checking feature Temp9am
Checking feature Temp3pm
Best feature is Pressure3pm
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Pressure9am
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MinTemp
Depth: 10
Depth: 10
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature Win

...

Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Rainfall
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Humidity3pm
Depth: 10
Depth: 10
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MaxTemp
Depth: 10
Depth: 10
Dep

Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MaxTemp
Depth: 10
Depth: 10
Depth: 9
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MaxTemp
Depth: 9
Depth: 9
Depth: 6
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pres

Checking feature Temp3pm
Best feature is MinTemp
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MinTemp
Depth: 10
Depth: 10
Depth: 9
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is WindSpeed9am
Depth: 9
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm


Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is MinTemp
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Temp3pm
Depth: 9
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Rainfall
Depth: 10
Depth: 10
Depth: 9
Checking feature MinTemp
Checking feature MaxTem

Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Humidity3pm
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking feature Temp3pm
Best feature is Pressure9am
Depth: 9
Depth: 9
Depth: 8
Checking feature MinTemp
Checking feature MaxTemp
Checking feature Rainfall
Checking feature WindGustSpeed
Checking feature WindSpeed9am
Checking feature WindSpeed3pm
Checking feature Humidity9am
Checking feature Humidity3pm
Checking feature Pressure9am
Checking feature Pressure3pm
Checking feature Temp9am
Checking f

In [30]:
y_train_hat, _ = dt.predict(X_train)
y_test_hat, _ = dt.predict(X_test)

In [75]:
# Accuracy metrics on train set

In [31]:
print(classification_report(y_train, y_train_hat))

              precision    recall  f1-score   support

          No       0.88      0.96      0.92     70355
         Yes       0.79      0.56      0.65     19985

    accuracy                           0.87     90340
   macro avg       0.84      0.76      0.79     90340
weighted avg       0.86      0.87      0.86     90340



In [76]:
# Accuracy metrics on test set

In [32]:
print(classification_report(y_test, y_test_hat))

              precision    recall  f1-score   support

          No       0.87      0.94      0.90     17551
         Yes       0.69      0.50      0.58      5034

    accuracy                           0.84     22585
   macro avg       0.78      0.72      0.74     22585
weighted avg       0.83      0.84      0.83     22585



In [77]:
# Implementation from sklearn

In [33]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=10)
clf = clf.fit(X_train, y_train)
y_hat_clf = clf.predict(X_test)
print(classification_report(y_test, y_hat_clf))

              precision    recall  f1-score   support

          No       0.87      0.94      0.90     17551
         Yes       0.69      0.50      0.58      5034

    accuracy                           0.84     22585
   macro avg       0.78      0.72      0.74     22585
weighted avg       0.83      0.84      0.83     22585



In [78]:
# The accuracy of the current implementation matches the accuracy of implementation from sklearn 